In [1]:
import torch

In [2]:
x = torch.randn(1,5)
print(x)

tensor([[ 1.5201,  2.0927, -1.1328, -0.6591,  0.9256]])


In [3]:
x.requires_grad_(True)
print(x.grad)
x.requires_grad

None


True

In [4]:
y = x.sum()
y

tensor(2.7466, grad_fn=<SumBackward0>)

In [5]:
error = y - 5
error

tensor(-2.2534, grad_fn=<SubBackward0>)

In [6]:
error.backward()

In [7]:
print(x.grad)

tensor([[1., 1., 1., 1., 1.]])


In [8]:
x.grad.zero_()

tensor([[0., 0., 0., 0., 0.]])

In [9]:
print(x.grad)

tensor([[0., 0., 0., 0., 0.]])


In [10]:
x

tensor([[ 1.5201,  2.0927, -1.1328, -0.6591,  0.9256]], requires_grad=True)

SOLO PARA CONJUNTO DE EVALUACIÓN

inputs = inputs(requires_grad=True)

for batch_idx, (inputs, targets) in enumerate(val_loader):

        inputs = torch.tensor(inputs, requires_grad=True)

	inputs.zero_grad_()
	
	outputs = net(inputs) 
	outputs = softmax(outputs)# Softmax!
	
	loss = criterion(outputs, targets)
	loss.backward()
				                               
      
        # https://discuss.pytorch.org/t/derivative-with-respect-to-the-input/49254/10
	inputs = inputs + 0.1*inputs.gradiente ### ==== optimizer.step()
	
	
	- ¿Sumar o restar? ¿0.1 o menor o mayor? argumentos
	
	- break cuando la maxima diferencia sea pequeña
	abs(max(outputs-targets)) < umbral
	
	- ¿Poner limite epochs?

	** ver primero que imagen resultante obtenemos


UNA VEZ 'EMPAREJADO' A LOS CONJUNTOS SEGMENTAR

------------------------------

In [89]:
x = torch.randn(1,5)
print("Initial x: {}".format(x))
print("Initial x sum: {}\n".format(x.sum()))


for k in range(3):
    
    print("\n---")
    
    x.requires_grad_(True)
    
    
    y = x.sum() ### FORWARD
    # https://discuss.pytorch.org/t/catrogircal-cross-entropy-with-soft-classes/50871
    error = (y - 5)**2  # CALCULATE LOSS CXE 
    
    
    error.backward()
    print("x grad: {}".format(x.grad))
    x = x.detach() - 0.1*x.grad
    
    #x.grad.detach_()
    #x.grad.zero_()
    
    print("y: {}".format(y))
    
    print("x: {}".format(x))
    print("x sum: {}".format(x.sum()))
    
    # x.grad = None  # no se mantiene debido al detach() en la asignacion de x
    

Initial x: tensor([[ 0.0896,  1.0414, -0.5398, -0.6193,  1.0945]])
Initial x sum: 1.0663421154022217


---
x grad: tensor([[-7.8673, -7.8673, -7.8673, -7.8673, -7.8673]])
y: 1.0663421154022217
x: tensor([[0.8763, 1.8281, 0.2469, 0.1674, 1.8812]])
x sum: 5.0

---
x grad: tensor([[0., 0., 0., 0., 0.]])
y: 5.0
x: tensor([[0.8763, 1.8281, 0.2469, 0.1674, 1.8812]])
x sum: 5.0

---
x grad: tensor([[0., 0., 0., 0., 0.]])
y: 5.0
x: tensor([[0.8763, 1.8281, 0.2469, 0.1674, 1.8812]])
x sum: 5.0


-------------------------

In [109]:
def CXE(predicted, target):
    return -(target * torch.log(predicted)).sum(dim=1).mean()

In [127]:
x = torch.randn(1,5)
x_original = x.detach()
target = torch.ones(1,5)*(1/5)
print("Initial x: {}".format(x))
print("Initial x sum: {}".format(x.sum()))
print("Initial y: {}".format(torch.nn.functional.softmax(x, dim=1)))
print("Target: {}\n".format(target))

umbral_salida = 0.02


for k in range(30):
    
    print("\n---")
    
    x.requires_grad_(True)
    
    y = torch.nn.functional.softmax(x, dim=1) # Simula salida softmax de modelo
    # https://discuss.pytorch.org/t/catrogircal-cross-entropy-with-soft-classes/50871
    error = CXE(y, target) # CALCULATE LOSS CXE
    
    
    error.backward()
    print(f"x grad: {x.grad}")
    x = x.detach() - 0.9*x.grad
    
    #x.grad.detach_()
    #x.grad.zero_()
    
    
    print("x: {}".format(x))
    #print("x sum: {}".format(x.sum()))
    print("y: {}".format(y))
    
    # x.grad = None  # no se mantiene debido al detach() en la asignacion de x
    if (y-target).abs().max()<umbral_salida: # Si la maxima diferencia es menor a un umbral salimos
        print(f"\n-----Salimos en la iteración {k}-----")
        break


print("x original",x_original)

Initial x: tensor([[-0.9923,  2.3752,  1.2397,  0.8197,  3.0838]])
Initial x sum: 6.5260443687438965
Initial y: tensor([[0.0096, 0.2779, 0.0893, 0.0587, 0.5645]])
Target: tensor([[0.2000, 0.2000, 0.2000, 0.2000, 0.2000]])


---
x grad: tensor([[-0.1904,  0.0779, -0.1107, -0.1413,  0.3645]])
x: tensor([[-0.8210,  2.3050,  1.3394,  0.9469,  2.7558]])
y: tensor([[0.0096, 0.2779, 0.0893, 0.0587, 0.5645]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[-0.1865,  0.1076, -0.0829, -0.1209,  0.2827]])
x: tensor([[-0.6531,  2.2082,  1.4140,  1.0557,  2.5013]])
y: tensor([[0.0135, 0.3076, 0.1171, 0.0791, 0.4827]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[-0.1819,  0.1159, -0.0572, -0.1002,  0.2235]])
x: tensor([[-0.4894,  2.1039,  1.4655,  1.1459,  2.3002]])
y: tensor([[0.0181, 0.3159, 0.1428, 0.0998, 0.4235]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[-0.1767,  0.1122, -0.0351, -0.0802,  0.1799]])
x: tensor([[-0.3304,  2.0029,  1.4971,  1.2181,  2.1383]])
y: tensor([[0.0233, 0.

--------------------------------------

In [132]:
x = torch.randn(1,5)
x_original = x.detach()
target = torch.ones(1,5)*(1/5) # [1, 0, 0] Target A parariamos cuando el maximo este en clase A
print("Initial x: {}".format(x))
print("Initial x sum: {}".format(x.sum()))
print("Initial y: {}".format(torch.nn.functional.softmax(x, dim=1)))
print("Target: {}\n".format(target))

umbral_salida = 0.02


for k in range(1000):
    
    print("\n---")
    
    x.requires_grad_(True)
    
    y = torch.nn.functional.softmax(x, dim=1) # Simula salida softmax de modelo
    # https://discuss.pytorch.org/t/catrogircal-cross-entropy-with-soft-classes/50871
    error = CXE(y, target) + torch.nn.L1Loss()(x_original, x)*0.5 # CALCULATE LOSS CXE + ¿L1?
    
    
    error.backward()
    print(f"x grad: {x.grad}")
    x = x.detach() - 0.9*x.grad
    
    #x.grad.detach_()
    #x.grad.zero_()
    
    
    print("x: {}".format(x))
    #print("x sum: {}".format(x.sum()))
    print("y: {}".format(y))
    
    # x.grad = None  # no se mantiene debido al detach() en la asignacion de x
    if (y-target).abs().max()<umbral_salida: # Si la maxima diferencia es menor a un umbral salimos
        print(f"\n-----Salimos en la iteración {k}-----")
        break

print("x original",x_original)

Initial x: tensor([[-1.2354, -0.8405, -1.6544, -0.1572,  0.3850]])
Initial x sum: -3.5025289058685303
Initial y: tensor([[0.0898, 0.1333, 0.0591, 0.2639, 0.4539]])
Target: tensor([[0.2000, 0.2000, 0.2000, 0.2000, 0.2000]])


---
x grad: tensor([[-0.1102, -0.0667, -0.1409,  0.0639,  0.2539]])
x: tensor([[-1.1362, -0.7804, -1.5276, -0.2148,  0.1565]])
y: tensor([[0.0898, 0.1333, 0.0591, 0.2639, 0.4539]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0080,  0.0542, -0.0270, -0.0286,  0.0934]])
x: tensor([[-1.1434, -0.8292, -1.5033, -0.1890,  0.0724]])
y: tensor([[0.1080, 0.1542, 0.0730, 0.2714, 0.3934]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0107,  0.0515, -0.0228, -0.0126,  0.0732]])
x: tensor([[-1.1530, -0.8755, -1.4828, -0.1777,  0.0065]])
y: tensor([[0.1107, 0.1515, 0.0772, 0.2874, 0.3732]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0126, -0.1514, -0.0190, -0.0013,  0.0591]])
x: tensor([[-1.1644, -0.7393, -1.4657, -0.1765, -0.0467]])
y: tensor([[0.1126, 0

x: tensor([[-1.1141, -0.8870, -1.3825, -0.2844, -0.2844]])
y: tensor([[0.1337, 0.1740, 0.0991, 0.2965, 0.2965]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0315, -0.1350,  0.0005,  0.0015,  0.0015]])
x: tensor([[-1.1425, -0.7655, -1.3830, -0.2857, -0.2857]])
y: tensor([[0.1315, 0.1650, 0.1005, 0.3015, 0.3015]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0257,  0.0833, -0.0012, -0.0039, -0.0039]])
x: tensor([[-1.1656, -0.8405, -1.3820, -0.2822, -0.2822]])
y: tensor([[0.1257, 0.1833, 0.0988, 0.2961, 0.2961]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0246, -0.1276,  0.0003,  0.0013,  0.0013]])
x: tensor([[-1.1877, -0.7257, -1.3823, -0.2834, -0.2834]])
y: tensor([[0.1246, 0.1724, 0.1003, 0.3013, 0.3013]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0197,  0.0901, -0.0014, -0.0042, -0.0042]])
x: tensor([[-1.2055, -0.8067, -1.3810, -0.2797, -0.2797]])
y: tensor([[0.1197, 0.1901, 0.0986, 0.2958, 0.2958]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[0.

x grad: tensor([[ 1.7291e-02,  8.3373e-02,  9.3415e-05, -3.7863e-04, -3.7863e-04]])
x: tensor([[-1.2389, -0.8515, -1.3819, -0.2851, -0.2851]])
y: tensor([[0.1173, 0.1834, 0.1001, 0.2996, 0.2996]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[-0.1828, -0.1273,  0.0016,  0.0042,  0.0042]])
x: tensor([[-1.0744, -0.7369, -1.3834, -0.2889, -0.2889]])
y: tensor([[0.1172, 0.1727, 0.1016, 0.3042, 0.3042]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0329,  0.0863, -0.0024, -0.0084, -0.0084]])
x: tensor([[-1.1040, -0.8146, -1.3812, -0.2814, -0.2814]])
y: tensor([[0.1329, 0.1863, 0.0976, 0.2916, 0.2916]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0308,  0.0747, -0.0009, -0.0023, -0.0023]])
x: tensor([[-1.1317, -0.8818, -1.3804, -0.2793, -0.2793]])
y: tensor([[0.1308, 0.1747, 0.0991, 0.2977, 0.2977]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0290, -0.1344,  0.0006,  0.0024,  0.0024]])
x: tensor([[-1.1578, -0.7608, -1.3809, -0.2815, -0.2815]])
y: tensor([[0.1290, 0

y: tensor([[0.1197, 0.1674, 0.1017, 0.3056, 0.3056]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[-1.8427e-01,  8.5669e-02, -3.1702e-05, -6.8405e-04, -6.8405e-04]])
x: tensor([[-1.0700, -0.8403, -1.3822, -0.2850, -0.2850]])
y: tensor([[0.1157, 0.1857, 0.1000, 0.2993, 0.2993]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0356,  0.0706, -0.0008, -0.0027, -0.0027]])
x: tensor([[-1.1020, -0.9038, -1.3815, -0.2826, -0.2826]])
y: tensor([[0.1356, 0.1706, 0.0992, 0.2973, 0.2973]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0331, -0.1377,  0.0006,  0.0020,  0.0020]])
x: tensor([[-1.1318, -0.7799, -1.3821, -0.2844, -0.2844]])
y: tensor([[0.1331, 0.1623, 0.1006, 0.3020, 0.3020]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0271,  0.0807, -0.0010, -0.0034, -0.0034]])
x: tensor([[-1.1562, -0.8525, -1.3812, -0.2813, -0.2813]])
y: tensor([[0.1271, 0.1807, 0.0990, 0.2966, 0.2966]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0258, -0.1296,  0.0004,  0.0017,  0.00

x: tensor([[-1.2193, -0.7798, -1.3819, -0.2858, -0.2858]])
y: tensor([[0.1220, 0.1643, 0.1018, 0.3059, 0.3059]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0178,  0.0828,  0.0001, -0.0004, -0.0004]])
x: tensor([[-1.2353, -0.8543, -1.3820, -0.2854, -0.2854]])
y: tensor([[0.1178, 0.1828, 0.1001, 0.2996, 0.2996]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0177, -0.1278,  0.0016,  0.0042,  0.0042]])
x: tensor([[-1.2512, -0.7393, -1.3835, -0.2892, -0.2892]])
y: tensor([[0.1177, 0.1722, 0.1016, 0.3042, 0.3042]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[-0.1861,  0.0901, -0.0002, -0.0019, -0.0019]])
x: tensor([[-1.0837, -0.8204, -1.3833, -0.2875, -0.2875]])
y: tensor([[0.1139, 0.1901, 0.0998, 0.2981, 0.2981]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0337,  0.0740, -0.0009, -0.0035, -0.0035]])
x: tensor([[-1.1141, -0.8870, -1.3825, -0.2844, -0.2844]])
y: tensor([[0.1337, 0.1740, 0.0991, 0.2965, 0.2965]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0

y: tensor([[0.1203, 0.1764, 0.1003, 0.3015, 0.3015]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0199, -0.1328,  0.0017,  0.0056,  0.0056]])
x: tensor([[-1.2343, -0.7645, -1.3822, -0.2858, -0.2858]])
y: tensor([[0.1199, 0.1672, 0.1017, 0.3056, 0.3056]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 1.5926e-02,  8.5445e-02, -9.6411e-06, -6.8063e-04, -6.8063e-04]])
x: tensor([[-1.2486, -0.8414, -1.3822, -0.2851, -0.2851]])
y: tensor([[0.1159, 0.1854, 0.1000, 0.2993, 0.2993]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[-0.1840, -0.1257,  0.0015,  0.0041,  0.0041]])
x: tensor([[-1.0831, -0.7283, -1.3835, -0.2888, -0.2888]])
y: tensor([[0.1160, 0.1743, 0.1015, 0.3041, 0.3041]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0317,  0.0878, -0.0025, -0.0085, -0.0085]])
x: tensor([[-1.1116, -0.8073, -1.3813, -0.2811, -0.2811]])
y: tensor([[0.1317, 0.1878, 0.0975, 0.2915, 0.2915]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0297,  0.0759, -0.0009, -0.0023, -0.00

x: tensor([[-1.2512, -0.7393, -1.3835, -0.2892, -0.2892]])
y: tensor([[0.1177, 0.1722, 0.1016, 0.3042, 0.3042]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[-0.1861,  0.0901, -0.0002, -0.0019, -0.0019]])
x: tensor([[-1.0837, -0.8204, -1.3833, -0.2875, -0.2875]])
y: tensor([[0.1139, 0.1901, 0.0998, 0.2981, 0.2981]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0337,  0.0740, -0.0009, -0.0035, -0.0035]])
x: tensor([[-1.1141, -0.8870, -1.3825, -0.2844, -0.2844]])
y: tensor([[0.1337, 0.1740, 0.0991, 0.2965, 0.2965]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0315, -0.1350,  0.0005,  0.0015,  0.0015]])
x: tensor([[-1.1425, -0.7655, -1.3830, -0.2857, -0.2857]])
y: tensor([[0.1315, 0.1650, 0.1005, 0.3015, 0.3015]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0.0257,  0.0833, -0.0012, -0.0039, -0.0039]])
x: tensor([[-1.1656, -0.8405, -1.3820, -0.2822, -0.2822]])
y: tensor([[0.1257, 0.1833, 0.0988, 0.2961, 0.2961]], grad_fn=<SoftmaxBackward>)

---
x grad: tensor([[ 0